© Jacob White 2025, All Rights Reserved

# An Introduction to Monte Carlo Integration in Python

## 1. Monte Carlo Integration

Perhaps the simplest way to introduce Monte Carlo methods is to demonstrate its application in numerical integration. Say we want to compute the integral $$I = \int_{-1}^1 e^{-x^2} \ dx \tag{1}$$ which cannot be computed using elementary methods of calculus. The gyst of the Monte Carlo approach is to sample $N$ points $x_1, \dots, x_N$ uniformly in $[-1, 1]$, so that (1) can be approximated by $$\boxed{I = Q_N := \frac{2}{N} \sum_{i = 1}^N f(x_i)} \tag{2}$$ where $f(x) = e^{-x^2}$. 

In [114]:
import scipy.stats as stats
import numpy as np
N = 100

# Draw samples from the uniform distribution on [-1, 1]
samples = stats.uniform.rvs(loc = -1, scale = 2, size = N)

# Define the function f(x)
def f(x):
    return np.exp(-x**2)

# Compute and print the Monte Carlo approximation
approx = 2*sum(map(f, samples))/N
print(f'Monte Carlo Approximation: {approx}')

from scipy.integrate import quad
# Compute using standard numerical quadrature
q, error = quad(f, -1, 1)
print(f'Quadrature Approximation: {q}, Quadrature Error: {error}')

# Difference
print(f'Difference between quadrature and Monte Carlo approximation:{q - approx}')

Monte Carlo Approximation: 1.5061745556732637
Quadrature Approximation: 1.493648265624854, Quadrature Error: 1.6582826951881447e-14
Difference between quadrature and Monte Carlo approximation:-0.012526290048409772


### 1.1 General Monte Carlo Integration

If we want to compute the multidimensional definite integral $$I = \int_\Omega f(\mathbf{x}) \ d \mathbf{x} \tag{3}$$ using Monte Carlo integration, the approach is similar to the one-dimensional case. We sample points $\mathbf{x}_1, \mathbf{x}_2, \dots, \mathbf{x}_N$ on $\Omega$ and compute $$\boxed{I \approx Q_N := \frac{V}{N} \sum_{i = 1}^N f(\mathbf{x}_i) = V \langle f \rangle\tag{4}}$$ where $V = \mathrm{Vol}(\Omega) = \int_\Omega \ d \mathbf{x}$. 

> **Example.** Let's approximate the integral $$\int_{-1}^1 \int_{-3}^5 x^2 + y^2 \ dx \ dy$$ using Monte Carlo integration (the exact answer is $\frac{320}{3} \approx 106.6$). The volume of $\Omega = [-1, 1] \times [-3, 5]$ is the product of the length of the constituant intervals: $V = 2 \times 8 = 16$. 

In [143]:
# Draw samples from the rectangle [-1, 1] x [-3, 5] by sampling x and y coordinates separately:
N = 100
V = 16
samples_x = stats.uniform.rvs(loc = -1, scale = 2, size = N)
samples_y = stats.uniform.rvs(loc = -3, scale = 8, size = N)
samples = np.array([samples_x, samples_y]).T

def f(x, y):
    return x**2 + y**2

# Compute the values of f

mcapprox = (V/N)*sum(map(lambda xy: f(*xy), samples))
print(f'Monte Carlo approximation: {mcapprox}. Error: {mcapprox - 320/3}')

Monte Carlo approximation: 102.3936341694203. Error: -4.273032497246376


### 1.2 Error Analysis for Monte Carlo Integration

In this section, we derive an estimation of the error of $Q_N$. Note that $$\mathrm{Var}(f) = E(\sigma_N^2) := \frac{1}{N - 1} \sum_{i = 1}^N E\left[(f(\mathbf{x}_i) - \langle f\rangle)^2\right]$$ and so $$\mathrm{Var}(Q_N) = \frac{V^2}{N^2} \sum_{i = 1}^N \mathrm{Var}(f) = \frac{V^2}{N} \mathrm{Var}(f) = \frac{V^2}{N} E(\sigma_N^2).$$ We now use the fact that the sequence $\{E(\sigma_1^2), E(\sigma_2^2)), E(\sigma_3^2), \dots\}$ is bounded due to its limit being $\mathrm{Var}(f)$, as long as this is assumed finite (?), this variance decreases asymptotically to zero as $1/N$. The estimation of the error of $Q_N$ is thus $$\boxed{\delta Q_N \approx \sqrt{\mathrm{Var}(Q_N)} = V \frac{\sqrt{\mathrm{Var}(f)}}{\sqrt{N}}}$$

### 1.3 Bonus: Approximating $\pi$ using Monte Carlo

Consider a square of side length $2r$ and a circle therin of radius $r$. The circle-square area ratio is $$\frac{\pi r^2}{4r^2} = \frac{\pi}{4}.$$ So, we can randomly sample points in the square of side length $2r$, and compute $$\pi \approx 4 \cdot \frac{\text{Number of points in the circle}}{\text{Total number of points}}.$$

In [235]:
N = 100000
r = 1
samples_x = stats.uniform.rvs(loc = -r, scale = 2*r, size = N)
samples_y = stats.uniform.rvs(loc = -r, scale = 2*r, size = N)

points_circ = 0
points_square = 0
for x, y in zip(samples_x, samples_y):
    if x**2 + y**2 <= r**2:
        points_circ += 1
    else:
        points_square += 1

result = 4*points_circ/N

print(f'Approximation to pi:{result:.10f} ')
print(f'Error:{result - np.pi}')

Approximation to pi:3.1432000000 
Error:0.0016073464102071
